In [180]:
!which python
!pwd

/d/zuev/prog/made/2_adv_ml/.venv/Scripts/python
/d/zuev/prog/made/2_adv_ml/made_adv_ml_02_23/homework3


In [181]:
import pandas as pd
import numpy as np
import random
import nltk
import string
from collections import Counter, defaultdict
from itertools import product
from tqdm.notebook import tqdm

In [182]:
import warnings
warnings.filterwarnings("ignore")

In [183]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [184]:
ALPHABET = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "

# Пункт 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:

○ подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно
просто опустить, а вот пробелы лучше оставить);

○ возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3
предложения, иначе вряд ли сработает), зашифруйте их посредством
случайной перестановки символов;

○ расшифруйте их таким частотным методом.

In [185]:
with open(f"./corpora/AnnaKarenina.txt", "r", encoding='utf-8') as f:
    corpus1 = f.read()
    
with open(f"./corpora/WarAndPeace.txt", "r", encoding='utf-8') as f:
    corpus2 = f.read()

In [186]:
def remove_punctuation(text):
    return "".join(filter(lambda x: x in ALPHABET, text))

In [187]:
def prepare_text(text):
    return remove_punctuation(text.lower())

In [188]:
corpus1 = prepare_text(corpus1)
corpus2 = prepare_text(corpus2)

In [189]:
# def calc_frequency(text):
#     freq = dict(Counter(text).most_common())
#     total_count = sum(freq.values())
#     return {ch: count / total_count  for ch, count in freq.items()}


def calc_frequency(text, ngram=1):
    text = text[:len(text) // ngram * ngram]
    freq = set("".join(n_gram) for n_gram in product(ALPHABET, repeat=ngram))
    freq = {token: 0 for token in freq}
    total_count = 0
    
    for i in range(0, len(text) + 1 - ngram):
        freq[text[i:i + ngram]] += 1
        total_count += 1
    
    freq = sorted(
        {token: count / total_count for token, count in freq.items()}.items(), 
        key=lambda x: x[1],
        reverse=True
    )
    
    return dict(freq)

In [199]:
corpus1_freq = calc_frequency(corpus1)
corpus2_freq = calc_frequency(corpus2)

corpus1_freq, corpus2_freq

({' ': 0.1687583883329221,
  'о': 0.0956448099370864,
  'е': 0.07281912177724592,
  'а': 0.06896409572666888,
  'н': 0.05779535618355954,
  'и': 0.05528364122698895,
  'т': 0.04984502748163621,
  'с': 0.044241518029873216,
  'л': 0.04176219330134749,
  'в': 0.039199242893142285,
  'р': 0.03314933720759722,
  'к': 0.028538735473718862,
  'д': 0.024517635890257197,
  'м': 0.02386688676317384,
  'у': 0.022454672820417385,
  'п': 0.0200778189951609,
  'я': 0.017928874592692882,
  'ь': 0.016403000395160783,
  'ы': 0.015437771599423571,
  'г': 0.015130947802853048,
  'б': 0.01455675739660303,
  'ч': 0.014050292012628656,
  'з': 0.01361626295682736,
  'ж': 0.009434390059615686,
  'й': 0.00875242853096182,
  'ш': 0.007107004946282279,
  'х': 0.006469800823654053,
  'ю': 0.005189503446025807,
  'э': 0.002955755537403941,
  'щ': 0.0023874542635257175,
  'ц': 0.0023515305560577677,
  'ф': 0.0010488544754167003,
  'ъ': 0.0002426322537179565,
  'ё': 1.825631035256469e-05},
 {' ': 0.1703737332543340

In [200]:
text_sample = """
Мистер Икс – загадочный артист цирка, которого никто не видел без маски, - будоражит сердца и вызывает волну слухов в парижском обществе. Красавица Теодора – молодая богатая вдова, к ногам которой готов упасть весь высший свет. Может ли что-то связывать этих двух персонажей? Одного лишь циркового таланта недостаточно, чтобы завоевать сердце блистательной светской дамы. Да и нужна ли ее любовь тому, кто избрал одиночество и решил навсегда скрыть лицо под маской?
"""

text_sample = prepare_text(text_sample)
text_sample

'мистер икс  загадочный артист цирка которого никто не видел без маски  будоражит сердца и вызывает волну слухов в парижском обществе красавица теодора  молодая богатая вдова к ногам которой готов упасть весь высший свет может ли чтото связывать этих двух персонажей одного лишь циркового таланта недостаточно чтобы завоевать сердце блистательной светской дамы да и нужна ли ее любовь тому кто избрал одиночество и решил навсегда скрыть лицо под маской'

In [201]:
def encrypt(text, freq, ngram=1):
    text = text[:len(text) // ngram * ngram]
    tokens = list(freq.keys())
    encrypt = {
        orig: enc for orig, enc in zip(
            tokens, 
            random.sample(tokens, k=len(tokens))
        )
    }
    
    return "".join([encrypt[text[i:i + ngram]] for i in range(0, len(text), ngram)])


def decrypt(text, freq, ngram=1):
    text = text[:len(text) // ngram * ngram]
    freq_enc = calc_frequency(text, ngram)
    
    decrypt = {
        enc: orig for enc, orig in zip(
            freq_enc.keys(), freq.keys()
        )
    }
    
    return "".join([decrypt[text[i:i + ngram]] for i in range(0, len(text), ngram)])


def calc_accuracy(text1, text2):
    result = 0
    
    for i in range(len(text2)):
        if text1[i] == text2[i]:
            result += 1
    
    result /= len(text1)
    
    return result

In [202]:
text_enc = encrypt(text_sample, corpus2_freq)
text_dec = decrypt(text_enc, corpus2_freq)

print("raw: ", text_sample, end="\n\n")
print("enc: ", text_enc, end="\n\n")
print("dec: ", text_dec, end="\n\n")
print(calc_accuracy(text_sample, text_dec))

raw:  мистер икс  загадочный артист цирка которого никто не видел без маски  будоражит сердца и вызывает волну слухов в парижском обществе красавица теодора  молодая богатая вдова к ногам которой готов упасть весь высший свет может ли чтото связывать этих двух персонажей одного лишь циркового таланта недостаточно чтобы завоевать сердце блистательной светской дамы да и нужна ли ее любовь тому кто избрал одиночество и решил навсегда скрыть лицо под маской

enc:  з сщрюэ всээгёоёжкъбяыэёющ сщэь ювёэвкщкюкокэб вщкэбрэт жраэхргэзёсв ээхижкюём щэсрюжьёэ этягятёрщэткабиэсаифктэтэуёю мсвкзэкхйрсщтрэвюёсёт ьёэщркжкюёээзкакжёеэхкоёщёеэтжктёэвэбкоёзэвкщкюкыэокщктэиуёсщчэтрсчэтясц ыэстрщэзкмрщэа эъщкщкэстегятёщчэлщ фэжтифэурюскбёмрыэкжбкокэа цчэь ювкткокэщёаёбщёэбржксщёщкъбкэъщкхяэгёткртёщчэсрюжьрэха сщёщрачбкыэстрщсвкыэжёзяэжёэ эбимбёэа эррэадхктчэщкзиэвщкэ гхюёаэкж бкърсщткэ эюрц аэбётсрожёэсвюящчэа ькэукжэзёсвкы

dec:  мтнеив ткн  баяалочргы аветне зтвка коеовояо рткео ри стлид уиб манкт  уьлов

# Пункт 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если
вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не
так уж прост: после буквы E, которая действительно выделяется частотой, дальше
он анализировал уже конкретные слова и пытался угадать, какими они могли бы
быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что
давайте просто сделаем следующий логический шаг:

○ подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

○ проведите тестирование аналогично п.1, но при помощи биграмм1

In [194]:
corpus1_freq = calc_frequency(corpus1, 2)
corpus2_freq = calc_frequency(corpus2, 2)

corpus1_freq, corpus2_freq

({'о ': 0.023217931722496688,
  'е ': 0.018438895173909228,
  'а ': 0.01834820242856327,
  'и ': 0.017856459296330302,
  ' н': 0.015993724533153203,
  ' с': 0.01586651912409653,
  ' в': 0.014464315054819054,
  'то': 0.014229338396422701,
  ' п': 0.01382946583739733,
  ' о': 0.013163993095573074,
  'я ': 0.010942020834597044,
  ' и': 0.010901385773370608,
  'на': 0.010850739175320267,
  'ст': 0.010396686534659647,
  'ь ': 0.010205878421074639,
  'не': 0.00976537080082283,
  'но': 0.009619320145979986,
  'ал': 0.009514493466294394,
  ' к': 0.00883429787619969,
  'го': 0.008279540953369207,
  'он': 0.008261873535444669,
  'по': 0.008068709766136389,
  ' т': 0.007750107329563892,
  'ко': 0.007740095792739987,
  'ни': 0.007659414584217931,
  'ка': 0.007629379973746217,
  'ла': 0.007525731121922262,
  'ов': 0.0072713203038089185,
  'ен': 0.0072395189515447505,
  ' д': 0.007156482087299423,
  'л ': 0.007082278932016365,
  'й ': 0.007056955632991194,
  'во': 0.0069332837075194295,
  'ро': 0.00

In [195]:
text_enc = encrypt(text_sample, corpus2_freq, 2)
text_dec = decrypt(text_enc, corpus2_freq, 2)

print("raw: ", text_sample, end="\n\n")
print("enc: ", text_enc, end="\n\n")
print("dec: ", text_dec, end="\n\n")
print(calc_accuracy(text_sample, text_dec))

raw:  мистер икс  загадочный артист цирка которого никто не видел без маски  будоражит сердца и вызывает волну слухов в парижском обществе красавица теодора  молодая богатая вдова к ногам которой готов упасть весь высший свет может ли чтото связывать этих двух персонажей одного лишь циркового таланта недостаточно чтобы завоевать сердце блистательной светской дамы да и нужна ли ее любовь тому кто избрал одиночество и решил навсегда скрыть лицо под маской

enc:  флудёцуллбёктшсдощъвоютщяраэ чиёвыажм вруюс ккнбврккхщктадткаидцывгынфбуззсвйбвчё ёцефажнфяхзэлачлуэрхамё цшёпыфыфшниядпм мьыуй удаязуыздектяшцзусощызёкышзщъьзъпжсдухзъоеъеажзсоосдмьм вруюйдс врыфутяптяуэхюкыяхчвцжё ая чышри чацкщврврё яизэлатяпмкгыпчзигзлёцфмязрийддкоос уфихкыиёвыъегаецухфэмгажарощудсусеоокщврйхщхмидилатяё ёцефхщ заэухнжачоойдхшчлгыгхтэдьбтъьулккдбязуфнфэкуфщоъекыврзчзуврулкпызткдкзвсехюхоецнфкцёаткязнпъмъьё тйсакыацащзлдкт япм 

dec:  ьн д ннельжеьено кчеояссузтара еор п  о аснагоыхо гой тохол глша укоя урс едр

Результат стал хуже :(

# Пункт 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают.
Основная часть задания — в том, как можно их улучшить:

○ предложите метод обучения перестановки символов в этом задании,
основанный на MCMC-сэмплировании, но по-прежнему работающий на
основе статистики биграмм;

○ реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Реализуею следующий алгоритм:

1. Начните со случайной перестановки символов.

2. Выберите два случайных символа из перестановки и поменяйте их местами, создав новую перестановку.

3. Вычислите логарифм правдоподобия, основанного на статистике биграмм в тексте, с использованием новой перестановки.

4. Сравните логарифм правдоподобия для новой перестановки с логарифмом правдоподобия для текущей перестановки.

5. Повторять шаги 2-4 до тех пор, пока не будет достигнуто максимальное количество итераций.

6. Возвратить найденную перестановку.

In [203]:
def permute_text(text, permutation):
    return "".join([permutation[ch] for ch in text])


def calc_log_likelihood(text, bigram_freq):
    log_likelihood = 0
    
    for i in range(len(text) - 1):
        bigram = text[i:i + 2]
        log_likelihood += np.log(bigram_freq[bigram])
        
    return log_likelihood


def mcmc(text, bigram_freq, iters):
    embeddings = {c: i for i, c in enumerate(ALPHABET)}
            
    permutation = dict(zip(list(ALPHABET), random.sample(list(ALPHABET), len(ALPHABET))))
    permutation_best = permutation.copy()
    
    log_likelihood = calc_log_likelihood(permute_text(text, permutation), bigram_freq)
    log_likelihood_best = log_likelihood
    
    for i in tqdm(range(iters)):
        ch1, ch2 = random.sample(list(ALPHABET), 2)
        permutation[ch1], permutation[ch2] = permutation[ch2], permutation[ch1]
        
        log_likelihood_current = calc_log_likelihood(permute_text(text, permutation), bigram_freq)
        
        if log_likelihood_current < log_likelihood:
            if np.exp(log_likelihood_current - log_likelihood) < random.random():
                permutation[ch1], permutation[ch2] = permutation[ch2], permutation[ch1]
            else:
                log_likelihood = log_likelihood_current
        else:
            log_likelihood = log_likelihood_current
            if log_likelihood_current > log_likelihood_best:
                permutation_best = permutation.copy()
                log_likelihood_best = log_likelihood_current
                
    return permute_text(text, permutation_best)

In [204]:
def calc_frequency(text, ngram=1):
    text = text[:len(text) // ngram * ngram]
    freq = set("".join(n_gram) for n_gram in product(ALPHABET, repeat=ngram))
    freq = {token: 0 for token in freq}
    total_count = 0
    
    for i in range(0, len(text) + 1 - ngram):
        freq[text[i:i + ngram]] += 1
        total_count += 1
    
    freq = sorted(
        {token: max(count, 1) / total_count for token, count in freq.items()}.items(), 
        key=lambda x: x[1],
        reverse=True
    )
    
    return dict(freq)


bigram_freq = calc_frequency(corpus1 + corpus2, 2)

In [205]:
text_dec = mcmc(text_enc, bigram_freq, 100000)

print("raw: ", text_sample, end="\n\n")
print("enc: ", text_enc, end="\n\n")
print("dec: ", text_dec, end="\n\n")
print(calc_accuracy(text_sample, text_dec))

  0%|          | 0/100000 [00:00<?, ?it/s]

raw:  мистер икс  загадочный артист цирка которого никто не видел без маски  будоражит сердца и вызывает волну слухов в парижском обществе красавица теодора  молодая богатая вдова к ногам которой готов упасть весь высший свет может ли чтото связывать этих двух персонажей одного лишь циркового таланта недостаточно чтобы завоевать сердце блистательной светской дамы да и нужна ли ее любовь тому кто избрал одиночество и решил навсегда скрыть лицо под маской

enc:  з сщрюэ всээгёоёжкъбяыэёющ сщэь ювёэвкщкюкокэб вщкэбрэт жраэхргэзёсв ээхижкюём щэсрюжьёэ этягятёрщэткабиэсаифктэтэуёю мсвкзэкхйрсщтрэвюёсёт ьёэщркжкюёээзкакжёеэхкоёщёеэтжктёэвэбкоёзэвкщкюкыэокщктэиуёсщчэтрсчэтясц ыэстрщэзкмрщэа эъщкщкэстегятёщчэлщ фэжтифэурюскбёмрыэкжбкокэа цчэь ювкткокэщёаёбщёэбржксщёщкъбкэъщкхяэгёткртёщчэсрюжьрэха сщёщрачбкыэстрщсвкыэжёзяэжёэ эбимбёэа эррэадхктчэщкзиэвщкэ гхюёаэкж бкърсщткэ эюрц аэбётсрожёэсвюящчэа ькэукжэзёсвкы

dec:  мистер икс  загадочный артист цирка которого никто не видел без маски  будор

# Пункт 4

Расшифруйте сообщение

In [221]:
text = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

# str1 = ''.join(set(data))

# convert_data = []
# for i in list(corpus1_freq.keys()):
#     if i in ALPHABET:
#         convert_data.append(i)
# str2 = ''.join(convert_data[:len(set(data))])

# data = data.translate(str.maketrans(str1, str2))
# decode_data = mcmc(data, bigram_freq, 1000000)

# decode_data

In [223]:
text = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

text = text.translate(str.maketrans("".join(set(text)), ALPHABET[:len(set(text))]))

In [225]:
text_dec = mcmc(text, bigram_freq, 1000000)
text_dec

  0%|          | 0/1000000 [00:00<?, ?it/s]

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'